In [ ]:
#import pacakage setting

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

In [2]:
import chromedriver_autoinstaller

In [ ]:
#위스키 증류소 수집 코드

In [14]:
chrome_options = webdriver.ChromeOptions()
chromedriver_autoinstaller.install()

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 3)

driver.get('https://www.whiskybase.com/whiskies/distilleries')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\bmsyg\PycharmProjects\pythonProject\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bmsyg\AppData\Local\Temp\ipykernel_22196\1100265356.py", line 10, in <module>
    html = driver.page_source
  File "C:\Users\bmsyg\PycharmProjects\pythonProject\venv\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 447, in page_source
    driver.find_elements_by_css_selector('.foo')
  File "C:\Users\bmsyg\PycharmProjects\pythonProject\venv\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 346, in execute
  File "C:\Users\bmsyg\PycharmProjects\pythonProject\venv\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 245, in check_response
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.57

In [334]:
distillery_name_list = []
country_list = []
whiskies_list = []
votes_list = []
rating_list = []
link_list = []

In [335]:
for i in range(len(soup.select(".clickable"))):
#for i in range(3):    
    distillery_name_list.append(soup.select(".clickable")[i].text)
    link_list.append(soup.select(".clickable")[i].a['href']) 
    country_list.append(soup.select(".data")[4*(i+1)].text)
    whiskies_list.append(soup.select(".data")[4*(i+1)+1].text)
    votes_list.append(soup.select(".data")[4*(i+1)+2].text)
    rating_list.append(soup.select(".data")[4*(i+1)+3].text)

In [336]:
data = {
    'distillery_name' : distillery_name_list,
    'country' : country_list,
    'whiskies' : whiskies_list,
    'votes' : votes_list,
    'rating' : rating_list,
    'link' : link_list
}

In [337]:
distillery_table = pd.DataFrame(data)

In [338]:
driver.close()

In [340]:
distillery_table.to_csv('distillery_table_2023_07_25_VOL2.csv',index=False)

In [3]:
#distillery_table = pd.read_csv('distillery_table_2023_07_25_VOL2.csv')

In [ ]:
#위스키가 존재하지않는 증류소 제거

In [4]:
distillery_table = distillery_table[distillery_table.whiskies.isna()==False]

In [ ]:
#위스키 정보 링크 수집

In [5]:
whikie_link_list = []

In [6]:
def scrap_whikies_info(table):
    from selenium import webdriver 
    from bs4 import BeautifulSoup
    chrome_options = webdriver.ChromeOptions()
    #driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

    chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 3)
    
    driver.get(table.link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    
    for i in range(len(soup.select(".photo.buttons"))):
        try:
            whikie_img_list.append(soup.select(".photo.buttons")[i].a['href'])
        except:
            whikie_img_list.append(None)
        aa = soup.select(".clickable")
        whikie_link_list.append(aa[i]['href'])        
    driver.close()
    

In [12]:
distillery_table.progress_apply(scrap_whikies_info,axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [7:05:07<00:00, 28.34s/it]


1171    None
1172    None
1173    None
1174    None
1175    None
        ... 
2128    None
2129    None
2130    None
2131    None
2132    None
Length: 900, dtype: object

In [14]:
data ={
    'whikie_link' : whikie_link_list
}

In [15]:
whikie_table = pd.DataFrame(data)

In [16]:
whikie_table.to_csv('whikie_table_1100_2000_2023_07_25_vol1.csv',index=False)

In [ ]:
#위스키 정보 수집

In [20]:
def scrap_whikies_detail_info(table):
    from selenium import webdriver 
    from bs4 import BeautifulSoup
    chrome_options = webdriver.ChromeOptions()
    #driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

    chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 3)
    
    driver.get(table.whikie_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [21]:
soup = whikie_table[:1].progress_apply(scrap_whikies_detail_info,axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.51s/it]


In [22]:
soup[0]

<html class="js" lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-72Z2P1G3BJ&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-4SHJFZ6C6R&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-5035325-1&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.clarity.ms/s/0.7.8/clarity.js"></script><script async="" src="https://www.clarity.ms/tag/cdpmv56i88"></script><script>document.documentElement.className += 'js'</script>
<link href="https://assets.whiskybase.com/assets/